In [ ]:
#Codigo Criado por Luiz Eduardo Soares Emidio da Silva durante o desenvolvimento de um PIBIC 
from __future__ import print_function

import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.callbacks import EarlyStopping
from keras.utils import np_utils
from keras import regularizers
from keras.models import Model

from keras import backend as K 
K.set_learning_phase(1)

import matplotlib.pyplot as plt
import matplotlib.style as sty

from sklearn.metrics import confusion_matrix
import numpy as np
import itertools

import os
import random as rn
import seaborn as sns

import tensorflow as tf



In [ ]:
#Configurações Arbritarias escolhidas para treinamento
batch_size = 128
num_classes = 10
epochs = 50

#Cada imagem do MNIST tem o tamanho de 28 por 28 pixels
img_rows, img_cols = 28, 28

In [ ]:
#Carregando imagens e plotando alguns exemplos 
(x_train, y_train), (x_test, y_test) = mnist.load_data()
 
y_train = np_utils.to_categorical(y_train, num_classes)
y_test = np_utils.to_categorical(y_test, num_classes)

x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
input_shape = (28, 28, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train  /= 255.0
x_test /= 255.0

# plot first 36 images in MNIST
fig, ax = plt.subplots(6, 6, figsize = (12, 12))
fig.suptitle('First 36 images in MNIST')
fig.tight_layout(pad = 0.3, rect = [0, 0, 0.9, 0.9])
for x, y in [(i, j) for i in range(6) for j in range(6)]:
    ax[x, y].imshow(x_train[x + y * 6].reshape((28, 28)), cmap = 'gray')
#    ax[x, y].set_title(y_train[x + y * 6])




In [ ]:
#Inserir estrutura da CNN nessa parte do codigo
model = Sequential()

model.add(Conv2D(28, (3,3), padding='same', kernel_regularizer=regularizers.l2(1e-4), input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(BatchNormalization())

model.add(Conv2D(28, (3,3), padding='same', kernel_regularizer=regularizers.l2(1e-4)))
model.add(Activation('relu'))
model.add(BatchNormalization())

model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

# -===========================================================================-

model.add(Conv2D(64, (3,3), padding='same', kernel_regularizer=regularizers.l2(1e-4), input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(BatchNormalization())

model.add(Conv2D(64, (3,3), padding='same', kernel_regularizer=regularizers.l2(1e-4)))
model.add(Activation('relu'))
model.add(BatchNormalization())

model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))

# -===========================================================================-

model.add(Conv2D(128, (3,3), padding='same', kernel_regularizer=regularizers.l2(1e-4), input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(BatchNormalization())

model.add(Conv2D(128, (3,3), padding='same', kernel_regularizer=regularizers.l2(1e-4)))
model.add(Activation('relu'))
model.add(BatchNormalization())

model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Flatten())

model.add(Dense(512,activation ='relu'))

model.add(Dense(num_classes, activation='softmax'))

model.summary()


In [ ]:
# Compilar a CNN

model.compile(loss=keras.losses.categorical_crossentropy,optimizer=keras.optimizers.Adadelta(),metrics=['accuracy'])

In [ ]:
# Imprimir a topologia

model.summary()

In [ ]:
# Treinar a CNN

es=EarlyStopping(monitor='vai_loss', mode='min', verbose=1)
results=model.fit(x_train, y_train, 
                  batch_size=batch_size,
                  epochs=epochs,
                  verbose=1,
                  validation_split=0.2,
                  callbacks=[es]
                  )

In [ ]:
# Executar a avaliação (teste)

score=model.evaluate(x_test, y_test, verbose=0)
print('Test loss: ', score[0])
print('Test accuracy: ', score[1])

In [ ]:
# Plota o grafico do histórico de evolução da taxa de acertos da rede

sty.use('seaborn-whitegrid')
plt.plot(results.history['accuracy'],'k--')
plt.plot(results.history['val_accuracy'],'k')
plt.ylabel('Accuracy')

plt.xlabel('Epoch')

plt.legend(['Train', 'Val'], loc='upper left')

plt.show()



# Plota o grafico do histórico de evolução da taxa de perda da rede 

plt.plot(results.history['loss'],'k--')

plt.plot(results.history['val_loss'],'k')

plt.ylabel('Loss')

plt.xlabel('Epoch')

plt.legend(['Train', 'Val'], loc='upper left')

plt.show()

In [ ]:

# Preparar Matriz de Confusão

y_predict=np.argmax(model.predict(x_test), axis=1)

# Preparar Matriz de Confusão

cm=confusion_matrix(np.argmax(y_test, axis=1), y_predict)

In [ ]:
# Plotar a matrix de confusão 


def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues):
  plt.imshow(cm, interpolation = 'nearest',cmap=cmap)
  plt.title(title)
  plt.colorbar()
  tick_marks = np.arange(len(classes))
  plt.xticks(tick_marks, classes, rotation = 45)
  plt.yticks(tick_marks, classes)
  
  if normalize:
    cm = cm.astype('float')/cm.sum(axis=1)[:,np.newaxis]
    print("Normalized confusion matrix")
  else:
    print("Confusion matrix, without normalization")
  
  print(cm)
  
  thresh = cm.max()/2
  for i, j in itertools.product(range(cm.shape[0]),range(cm.shape[1])):
    plt.text(j, i, cm[i,j], horizontalalignment="center", color="white" if cm[i,j]>thresh else "black")
   
  plt.tight_layout()
  plt.ylabel('True label')
  plt.xlabel('Predicted label')

In [ ]:
# Chamada da matrix de confuzão 

cm_plot_labels = ['0','1','2','3','4','5','6','7','8','9']

plot_confusion_matrix(cm, cm_plot_labels, title='Consioin Matrix')



In [ ]:
# Salvando o modelo, a arquitetura da CNN

model_json = model.to_json()
with open("cnn.json", "w") as json_file:
  json_file.write(model_json)

# Salvando os pessos da CNN

model.save_weights("cnn_weights.h5")

In [ ]:
from keras.models import model_from_json

# Carregando o modelo
json_file=open("cnn.json", "r")
cnn_json=json_file.read()
json_file.close()

cnn_model=model_from_json(cnn_json)

# Carregando os pesos

cnn_model.load_weights("cnn_weights.h5")

# Predição usando o modelo carregando
y_predict=np.argmax(cnn_model.predict(x_test), axis=1)

print(y_predict)
y=np.argmax(y_test, axis=1)
print(y)

print(x_test.shape)
print(x_test[0])